# stat 128 final project

In [10]:
import pandas as pd 
import numpy as np 
import os
import requests
import urllib
from PIL import Image

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics

import tensorflow as tf 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19

In [11]:

path_train = "./DATASET/TRAIN"
path_test = "./DATASET/TEST"
path_to_model = './models'
model_name = "whatever.hdf5"# 'model_simple4.hdf5'

In [12]:
def get_numpy_image_array(path, img_size = (128,128)):
    image_names_o = os.listdir(path + "/O")
    image_names_r = os.listdir(path + "/R")
    numpy_img_array_o = []
    numpy_img_array_r = []

    # for the images in O category 
    for img_name in image_names_o:
        # some images in grey scale
        img = Image.open(path + "/O/" + img_name).convert("RGB")
        img = img.resize(img_size)
        img_array = np.asarray(img)
        if img.size == (128,128):
            numpy_img_array_o.append(img_array)

    # for the images in R category 
    for img_name in image_names_r:
        # some images in grey scale
        img = Image.open(path + "/R/" + img_name).convert("RGB")
        img = img.resize(img_size)
        img_array = np.asarray(img)
        if img.size == (128,128):
            numpy_img_array_r.append(img_array)



    return np.asarray(numpy_img_array_o, dtype="float64"), np.asarray(numpy_img_array_r, dtype="float64")

## getting images into a numpy array for training/test and organic/recycle

In [13]:

organic_images_train, recyclable_images_train = get_numpy_image_array(path_train)
organic_images_test, recyclable_images_test = get_numpy_image_array(path_test)

In [14]:
x_train = np.concatenate([organic_images_train,recyclable_images_train])
x_test = np.concatenate([organic_images_test,recyclable_images_test])

In [15]:
y_organic_train = np.repeat("O",len(organic_images_train))
y_recycle_train = np.repeat("R",len(recyclable_images_train))


In [16]:
y_organic_test = np.repeat("O",len(organic_images_test))
y_recycle_test = np.repeat("R",len(recyclable_images_test))

In [17]:
y_train = np.concatenate([y_organic_train,y_recycle_train])
y_test = np.concatenate([y_organic_test,y_recycle_test])

confirm size 

In [18]:
le_1 = LabelEncoder()
y_train = le_1.fit_transform(y_train)
y_train = tf.keras.utils.to_categorical(y_train)

le_2 = LabelEncoder()
y_test = le_2.fit_transform(y_test)
y_test = tf.keras.utils.to_categorical(y_test)



In [19]:
x_train = x_train / 255
x_test = x_test / 255

In [20]:
print(x_train.shape)
print(y_train.shape)

print(x_test.shape)
print(y_test.shape)

(22564, 128, 128, 3)
(22564, 2)
(2513, 128, 128, 3)
(2513, 2)


In [21]:
# early stopping

monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=2,verbose=2,mode='auto')
checkpointer = ModelCheckpoint(filepath=path_to_model+model_name,verbose=0,save_best_only=True)
# checkpointer256 = ModelCheckpoint(filepath='/content/drive/My Drive/180/final project/dnn/modelo256.hdf5',verbose=0,save_best_only=True)






In [22]:
# training!!
for i in range(5):
    model_simple.fit(x_train ,y_train,
                    batch_size = 20,
                    epochs = 20,
                    verbose=2,
                    callbacks=[monitor, checkpointer],
                    validation_data=(x_test,y_test))



NameError: name 'model_simple' is not defined

In [ ]:
y_true = np.argmax(y_test,axis = 1)
y_pred = model_simple.predict(x_test)
y_pred = np.argmax(y_pred,axis = 1)
report = metrics.classification_report(y_true,y_pred)

f1_simple = metrics.f1_score(y_true,y_pred,average='weighted')
f1_simple

In [23]:
vgg19_128 = VGG19(weights='imagenet',include_top=False,input_shape=(128,128,3))

model_19_128 = Sequential()
for layer in vgg19_128.layers:
  model_19_128.add(layer)
for layer in model_19_128.layers:
  layer.trainable = False


model_19_128.add(Flatten())
model_19_128.add(Dense(2,activation='softmax'))






80142336/80134624 [==============================] - 75s 1us/step


In [24]:
checkpointer_vgg19_128 = ModelCheckpoint(filepath=path_to_model + '/' + model_name,verbose=0,save_best_only=True)
monitor = EarlyStopping(monitor='val_loss',min_delta=1e-3,patience=2,verbose=2,mode='auto')
model_19_128.compile(optimizer='adam',loss='categorical_crossentropy')

In [28]:


# for i in range(5):
model_19_128.fit(x_train,y_train,
            batch_size=32,
            epochs=20,
            verbose=2,
            callbacks=[monitor,checkpointer_vgg19_128],
            validation_data=(x_test,y_test))



KeyboardInterrupt: 

In [30]:
y_true = np.argmax(y_test,axis = 1)
y_pred = model_19_128.predict(x_test)
y_pred = np.argmax(y_pred,axis = 1)
f1_vgg19 = metrics.f1_score(y_true,y_pred,average="weighted")

In [31]:
f1_vgg19

0.8738099743838015

In [32]:
cf=metrics.classification_report(y_true, y_pred)
print(cf)

              precision    recall  f1-score   support

           0       0.86      0.93      0.89      1401
           1       0.90      0.81      0.85      1112

    accuracy                           0.87      2513
   macro avg       0.88      0.87      0.87      2513
weighted avg       0.88      0.87      0.87      2513

